In [ ]:
!pip install datasets
!pip install jsonlines
!pip install bert_score
!pip install git+https://github.com/google-research/bleurt.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-m_e234or
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-m_e234or
  Resolved https://github.com/google-research/bleurt.git to c

In [ ]:
#LLAMA
import numpy as np
import csv
import torch
import datasets
import random
import jsonlines

filename_test = "ecqa_test.jsonl"
filename_train = "ecqa_train.jsonl"
test_data_question = []
test_data_answer = []
test_data_choices = []
test_data_exp = []
with jsonlines.open(filename_test, 'r') as file:
    for line in file:
        test_data_question.append(line['question'])
        test_data_answer.append(line['answer'])
        test_data_choices.append(', '.join(line['choices']))
        test_data_exp.append(line['explanation'])
random.seed(3)
count = 0
prompts = []
idx = 0
prompt_index = random.sample(range(len(test_data_exp)),300)


test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0

L = []
bertscores = []
bleurtscores = []
meteorscores = []
bertscore_metric = datasets.load_metric("bertscore")
bleurtscore_metric = datasets.load_metric("bleurt","BLEURT-20")
meteor_metric = datasets.load_metric("meteor")
with open('llama-ecqa.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_question[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label.lower() == test_data_answer[index].lower()):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = 0
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = 0
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if("========" in line[:-1] and flag == 2):
            for choice in test_data_choices[index].split(', '):
                if(choice.lower() in exp):
                    label = choice.lower()
                    flag = 3
                    break
            if(flag == 3):
              test_dict[test_data_question[index]] = label+"#####"+exp
              l = []
              l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
              L.append(l)
              true_exp = test_data_exp[index]
              pred_exp = exp
              if (label.lower() == test_data_answer[index].lower()):
                  accuracy += 1
                  if(pred_exp != ''):
                      score = 0
                      score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                      bertscores.append(score)
                      score = 0
                      score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                      bleurtscores.append(score)
                      score = 0
                      score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                      meteorscores.append(score)
              else:
                  bertscores.append(0)
                  bleurtscores.append(0)
                  meteorscores.append(0)
              flag = 0
              count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: ' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            label = label.strip()
            if(label.lower() in test_data_choices[index].split(', ')):
                flag = 3
            else:
                print(label+"#"+test_data_choices[index]+"#"+test_data_answer[index])

import csv
with open('llama_ecqa.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Question","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_llama_ecqa.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_llama_ecqa.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_llama_ecqa.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []
print(accuracy/count)

Computing checksums: 100%|##########| 1/1 [00:07<00:00,  7.22s/it]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


When one increases the speed of a moving object, it gains speed.#sleeping, momentum, stress, excitement, working#working
work#sleeping, momentum, stress, excitement, working#working
severity of each wound#battle, injured person, body, food, emergency room#injured person
horse, that wins a single race, can go to a championship goal.#charriott, kentucky derby, race track, painting, triple crown#kentucky derby
derby#charriott, kentucky derby, race track, painting, triple crown#kentucky derby
sleep#deep sleep, get to sleep, conscious, open eyes, field#deep sleep
store is usually found in a shopping square or a mall but in this case the privately owned shop was just right there in the small village.#village, town, buy fishing pole, jerseys, city#town
running#moving fast, accidents, injuries, shortness of breath, losing#injuries
needs more space when they have more kids.#compliments, understand themselves, life partner, second chances, larger house#larger house
needs more space when they hav

In [ ]:
#LLAMA2
import numpy as np
import csv
import torch
import datasets
import random
import jsonlines

filename_test = "ecqa_test.jsonl"
filename_train = "ecqa_train.jsonl"
test_data_question = []
test_data_answer = []
test_data_choices = []
test_data_exp = []
with jsonlines.open(filename_test, 'r') as file:
    for line in file:
        test_data_question.append(line['question'])
        test_data_answer.append(line['answer'])
        test_data_choices.append(', '.join(line['choices']))
        test_data_exp.append(line['explanation'])
random.seed(3)
count = 0
prompts = []
idx = 0
prompt_index = random.sample(range(len(test_data_exp)),300)


test_dict = {}
L = []
count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
with open('llama-2-ecqa.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_question[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label.lower() == test_data_answer[index].lower()):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = 0
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = 0
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if("========" in line[:-1] and flag == 2):
            for choice in test_data_choices[index].split(', '):
                if(choice.lower() in exp):
                    label = choice.lower()
                    flag = 3
                    break
            if(flag == 3):
                test_dict[test_data_question[index]] = label+"#####"+exp
                l = []
                l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
                L.append(l)
                true_exp = test_data_exp[index]
                pred_exp = exp
                if (label.lower() == test_data_answer[index].lower()):
                    accuracy += 1
                    if(pred_exp != ''):
                        score = 0
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                        bertscores.append(score)
                        score = 0
                        score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                        bleurtscores.append(score)
                        score = 0
                        score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                        meteorscores.append(score)
                else:
                    bertscores.append(0)
                    bleurtscores.append(0)
                    meteorscores.append(0)
                flag = 0
                count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: ' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            label = label.strip()
            if(label.lower() in test_data_choices[index].split(', ')):
                flag = 3
            else:
                print(label+"#"+test_data_choices[index]+"#"+test_data_answer[index])

import csv
with open('llama2_ecqa.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Question","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_llama2_ecqa.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_llama2_ecqa.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_llama2_ecqa.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

print(accuracy/count)

seat belt#windows, trunks, four wheels, seat belts, diesel engines#seat belts
sleepiness#sleeping, momentum, stress, excitement, working#working
injury#battle, injured person, body, food, emergency room#injured person
rewards#a treat, guard house, fleas, punishment, abandoned#punishment
Triple Crown Races#charriott, kentucky derby, race track, painting, triple crown#kentucky derby
loss of energy#moving fast, accidents, injuries, shortness of breath, losing#injuries
pregnant#compliments, understand themselves, life partner, second chances, larger house#larger house
playground#office, church, bowling alley, building, playround#church
death#own death, die, great sorrow, fear, banquet#own death
optimistic#enthusiastic, splendid, glorious, cheerful, excited#enthusiastic
barber#art supplies, ladies bathroom, shower, hair salon, make person sick#hair salon
the wild#garden, the zoo, utah, south carolina, bermuda#utah
attic or loft#bungalow, garage, basement, attic, loft#attic
departmental stor

In [ ]:
#VICUNA

import numpy as np
import csv
import torch
import datasets
import random
import jsonlines

filename_test = "ecqa_test.jsonl"
filename_train = "ecqa_train.jsonl"
test_data_question = []
test_data_answer = []
test_data_choices = []
test_data_exp = []
with jsonlines.open(filename_test, 'r') as file:
    for line in file:
        test_data_question.append(line['question'])
        test_data_answer.append(line['answer'])
        test_data_choices.append(', '.join(line['choices']))
        test_data_exp.append(line['explanation'])
random.seed(3)
count = 0
prompts = []
idx = 0
prompt_index = random.sample(range(len(test_data_exp)),300)


test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
with open('vicuna-ecqa.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_question[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label.lower() == test_data_answer[index].lower()):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = 0
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = 0
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if("========" in line[:-1] and flag == 2):
            for choice in test_data_choices[index].split(', '):
                if(choice.lower() in exp):
                    label = choice.lower()
                    flag = 3
                    break
            if(flag == 3):
                test_dict[test_data_question[index]] = label+"#####"+exp
                l = []
                l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
                L.append(l)
                true_exp = test_data_exp[index]
                pred_exp = exp
                if (label.lower() == test_data_answer[index].lower()):
                    accuracy += 1
                    if(pred_exp != ''):
                        score = 0
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                        bertscores.append(score)
                        score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                        bleurtscores.append(score)
                        score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                        meteorscores.append(score)
                else:
                    bertscores.append(0)
                    bleurtscores.append(0)
                    meteorscores.append(0)
                flag = 0
                count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: ' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            label = label.strip()
            if(label.lower() in test_data_choices[index].split(', ')):
                flag = 3
            else:
                print(label+"#"+test_data_choices[index]+"#"+test_data_answer[index])
print(accuracy/count)
import csv
with open('vicuna_ecqa.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Question","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_vicuna_ecqa.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []


file = open('bleurt_scores_vicuna_ecqa.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_vicuna_ecqa.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []


grades the severity of the wound so that the doctor can treat each person in the best order.#battle, injured person, body, food, emergency room#injured person
it. The owner had never given the dog a command.#a treat, guard house, fleas, punishment, abandoned#punishment
horses is the Triple Crown. In order to win the Triple Crown, a horse must win the Kentucky Derby, the Preakness Stakes, and the Belmont Stakes.#charriott, kentucky derby, race track, painting, triple crown#kentucky derby
in a deep sleep.#deep sleep, get to sleep, conscious, open eyes, field#deep sleep
exhaustion#moving fast, accidents, injuries, shortness of breath, losing#injuries
to accommodate more people.#compliments, understand themselves, life partner, second chances, larger house#larger house
life. So, he decided to end his life by committing suicide.#suicide, grief, going to jail, not living, wrote his congressman#suicide
use the electricity in the RV, John would need to plug it in to an electrical outlet. A res

In [ ]:

#GPT3.5
import numpy as np
import csv
import torch
import datasets
import random
import jsonlines

filename_test = "ecqa_test.jsonl"
filename_train = "ecqa_train.jsonl"
test_data_question = []
test_data_answer = []
test_data_choices = []
test_data_exp = []
with jsonlines.open(filename_test, 'r') as file:
    for line in file:
        test_data_question.append(line['question'])
        test_data_answer.append(line['answer'])
        test_data_choices.append(', '.join(line['choices']))
        test_data_exp.append(line['explanation'])
random.seed(3)
count = 0
prompts = []
idx = 0
prompt_index = random.sample(range(len(test_data_exp)),300)


test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
with open('gpt-35-ecqa.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_question[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label.lower() == test_data_answer[index].lower()):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if("========" in line[:-1] and flag == 2):
            for choice in test_data_choices[index].split(', '):
                if(choice.lower() in exp):
                    label = choice.lower()
                    flag = 3
                    break
            if(flag == 3):
                test_dict[test_data_question[index]] = label+"#####"+exp
                l = []
                l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
                L.append(l)
                true_exp = test_data_exp[index]
                pred_exp = exp
                if (label.lower() == test_data_answer[index].lower()):
                    accuracy += 1
                    if(pred_exp != ''):
                        score = 0
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                        bertscores.append(score)
                        score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                        bleurtscores.append(score)
                        score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                        meteorscores.append(score)
                else:
                    bertscores.append(0)
                    bleurtscores.append(0)
                    meteorscores.append(0)
                flag = 0
                count+=1

        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: ' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            label = label.strip()
            if(label.lower() in test_data_choices[index].split(', ')):
                flag = 3
#             else:
#                 print(label+"#"+test_data_choices[index]+"#"+test_data_answer[index])
print(accuracy/count)
import csv
with open('gpt35_ecqa.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Question","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_gpt35_ecqa.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []

file = open('bleurt_scores_gpt35_ecqa.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_gpt35_ecqa.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.6872852233676976


In [ ]:
#GPT4
import numpy as np
import csv
import torch
import datasets
import random
import jsonlines

filename_test = "ecqa_test.jsonl"
filename_train = "ecqa_train.jsonl"
test_data_question = []
test_data_answer = []
test_data_choices = []
test_data_exp = []
with jsonlines.open(filename_test, 'r') as file:
    for line in file:
        test_data_question.append(line['question'])
        test_data_answer.append(line['answer'])
        test_data_choices.append(', '.join(line['choices']))
        test_data_exp.append(line['explanation'])
random.seed(3)
count = 0
prompts = []
idx = 0
prompt_index = random.sample(range(len(test_data_exp)),300)


test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
with open('gpt4-ecqa.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_question[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label.lower() == test_data_answer[index].lower()):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)
            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if("========" in line[:-1] and flag == 2):
            for choice in test_data_choices[index].split(', '):
                if(choice.lower() in exp):
                    label = choice.lower()
                    flag = 3
                    break
            if(flag == 3):
                test_dict[test_data_question[index]] = label+"#####"+exp
                l = []
                l.extend([test_data_question[index],test_data_answer[index],test_data_exp[index], label, exp])
                L.append(l)
                true_exp = test_data_exp[index]
                pred_exp = exp
                if (label.lower() == test_data_answer[index].lower()):
                    accuracy += 1
                    if(pred_exp != ''):
                        score = 0
                        score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                        bertscores.append(score)
                        score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                        bleurtscores.append(score)
                        score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                        meteorscores.append(score)
                else:
                    bertscores.append(0)
                    bleurtscores.append(0)
                    meteorscores.append(0)
                flag = 0
                count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Explanation: ' in line and flag == 1):
            exp = line.split('Explanation: ')[1][:-1]
            flag = 2
        elif ('Answer: ' in line and flag == 2):
            label = line.split('Answer: ')[1][:-1]
            label = label.strip()
            if(label.lower() in test_data_choices[index].split(', ')):
                flag = 3
            else:
                print(label+"#"+test_data_choices[index]+"#"+test_data_answer[index])
import csv
with open('gpt4_ecqa.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Question","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_gpt4_ecqa.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_gpt4_ecqa.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_gpt4_ecqa.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []
print(accuracy/count)

None of the above#sinks, cross river, collide, bottom out, carry people#bottom out
0.862876254180602


In [ ]:
scores = []
file = open('bert_scores_llama_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_llama2_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_vicuna_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_gpt35_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_gpt4_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

50.67401741867635
51.57859398394215
57.805146195671774
60.688041514137765
76.61594034836045


In [ ]:
scores = []
file = open('bleurt_scores_llama_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_llama2_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_vicuna_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_gpt35_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_gpt4_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))


31.855254238517723
33.987804669506694
34.656613902612165
39.57124625899128
50.55276697694657


In [ ]:

scores = []
file = open('meteor_scores_llama_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_llama2_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_vicuna_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_gpt35_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_gpt4_ecqa.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

17.19564148066468
14.396857736179946
18.691430436745552
25.14907494408426
28.13039879959417
